In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
import numpy as np
from PIL import Image

# define a função de crescimento da borda da região
def grow_edge(image, seed, threshold):
    """
    Cresce a borda de uma região na imagem a partir da semente (seed) usando conexão simples.
    Retorna uma máscara binária com os pixels da borda segmentada.
    """
    # cria uma matriz de zeros para a máscara binária
    segmented = np.zeros_like(image, dtype=bool)

    # define a lista de pixels da borda
    edge_pixels = [seed]

    # itera sobre a lista de pixels da borda até que todos os pixels sejam adicionados
    while edge_pixels:
        # remove o próximo pixel da lista
        pixel = edge_pixels.pop()

        # verifica se o pixel já foi adicionado à borda
        if segmented[pixel]:
            continue

        # obtém os pixels vizinhos do pixel
        neighbors = get_neighbors(pixel, image.shape[:2])

        # verifica se pelo menos um vizinho está dentro da região segmentada
        if np.any(segmented[neighbors]):
            # obtém a intensidade do pixel
            intensity = image[pixel]

            # obtém a intensidade média dos pixels vizinhos dentro da região segmentada
            mean_intensity = np.mean(image[segmented][0])

            # verifica se o pixel satisfaz o critério de conexão simples
            if np.abs(intensity - mean_intensity) <= threshold:
                # adiciona o pixel à borda
                segmented[pixel] = True

                # adiciona os pixels vizinhos à lista de pixels da borda
                edge_pixels.extend(neighbors)

    return segmented

# define a função para obter os pixels vizinhos de um pixel
def get_neighbors(pixel, image_shape):
    """
    Retorna uma lista com os pixels vizinhos do pixel fornecido dentro dos limites da imagem.
    """
    neighbors = []
    x, y = pixel

    # verifica os vizinhos acima e abaixo do pixel
    if x > 0:
        neighbors.append((x - 1, y))
    if x < image_shape[0] - 1:
        neighbors.append((x + 1, y))

    # verifica os vizinhos à esquerda e à direita do pixel
    if y > 0:
        neighbors.append((x, y - 1))
    if y < image_shape[1] - 1:
        neighbors.append((x, y + 1))

    return neighbors

# carrega a imagem
img = Image.open('imagem.jpg')

# converte a imagem para um array numpy
img_array = np.array(img)

# extrai os componentes de cor R, G, B da imagem
red_channel = img_array[:, :, 0]
green_channel = img_array[:, :, 1]
blue_channel = img_array[:, :, 2]

# define a região segmentada para crescer a borda
segmented = np.zeros_like

# cria uma matriz de zeros para a máscara binária
segmented = np.zeros_like(image, dtype=bool)

# define a lista de borda da região
region_boundary = []

# obtém a intensidade média do pixel da semente
seed_intensity = image[seed]

# itera sobre os pixels da borda até que todos os pixels sejam adicionados
while boundary_pixels:
    # remove o próximo pixel da lista
    pixel = boundary_pixels.pop()

    # verifica se o pixel já foi adicionado à borda
    if segmented[pixel]:
        continue

    # obtém a intensidade do pixel
    intensity = image[pixel]

    # verifica se o pixel satisfaz o critério de conexão simples com a região
    if np.abs(intensity - seed_intensity) <= threshold:
        # adiciona o pixel à borda
        region_boundary.append(pixel)

        # adiciona os pixels vizinhos à lista de pixels da borda
        neighbors = get_neighbors(pixel, image.shape[:2])
        boundary_pixels.extend(neighbors)

    # marca o pixel como adicionado à borda
    segmented[pixel] = True

# retorna a máscara binária da borda da região
return segmented, region_boundary


In [ ]:
import cv2
import numpy as np

# Read image
img = cv2.imread("capitol.tiff")

# Convert image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Compute edges using Canny edge detector
edges = cv2.Canny(gray, 50, 150)

# Dilate edges to connect nearby edges
kernel = np.ones((3,3), np.uint8)
dilated_edges = cv2.dilate(edges, kernel, iterations=1)

# Find contours of connected edges
contours, hierarchy = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image
cv2.drawContours(img, contours, -1, (0, 255, 0), 2)

# Display the results
cv2.imshow("Original Image", img)
cv2.imshow("Edge Growth Segmentation", dilated_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [12]:
import numpy as np
import matplotlib.pyplot as plt

def grow_edge(image, seed, n_sigma):
    """
    Realiza a segmentação de crescimento de borda a partir de uma semente
    em uma imagem colorida (RGB), utilizando a média e o desvio padrão das
    intensidades na região para o cálculo do limite de crescimento da borda.
    
    Parâmetros:
    - image: array numpy com a imagem (RGB).
    - seed: tupla com as coordenadas (linha, coluna) da semente.
    - n_sigma: número de desvios padrão a ser utilizado como limite de crescimento da borda.
    
    Retorna:
    - mask: array numpy com a máscara da borda segmentada.
    """
    # Converte a imagem para escala de cinza
    gray = np.dot(image[...,:3], [0.299, 0.587, 0.114])
    
    # Cria uma matriz de zeros para a máscara
    mask = np.zeros_like(gray, dtype=bool)
    
    # Define a semente na máscara
    mask[seed] = True
    
    # Inicia o crescimento da borda
    while True:
        # Obtém os pixels da borda atual
        border = np.logical_and(mask, np.logical_not(mask[1:-1, 1:-1]))
        
        # Para se não há mais pixels de borda
        if not np.any(border):
            break
        
        # Obtém os pixels da próxima borda
        next_border = np.zeros_like(border)
        for i in range(-1, 2):
            for j in range(-1, 2):
                if i == 0 and j == 0:
                    continue
                next_border[1:-1, 1:-1] |= np.logical_and(
                    border[1+i:-1+i, 1+j:-1+j],
                    np.abs(gray[1:-1, 1:-1] - np.mean(gray[border])) < n_sigma * np.std(gray[border])
                )
        
        # Atualiza a máscara com os pixels da próxima borda
        mask |= next_border
    
    return mask

In [20]:
import matplotlib.pyplot as plt

# Carrega a imagem
image = plt.imread('capitol.tiff')

# Define a semente (linha, coluna)
seed = (300, 400)

# Define o limite de crescimento em número de desvios padrão
n_sigma = 2

# Chama a função grow_edge para segmentar a borda
mask = grow_edge(image, seed, n_sigma)

# Mostra a imagem original e a máscara da borda segmentada
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(image)
ax[0].scatter(seed[1], seed[0], s=10, c='red')
ax[0].set_title('Imagem original')
ax[1].imshow(mask, cmap='gray')
ax[1].set_title('Máscara da borda segmentada')
plt.show()


ValueError: operands could not be broadcast together with shapes (492,780) (490,778) 